# Data Wrangling Criminal Expungement Dashboard Data

## Part 1: Setup

In [9]:
# STOP!
# Before you do anything, copy the code below. You'll need to shut down the 
# notebook, close and re-open the terminal and paste this code in *before* you call jupyter notebook.
# We need to switch to Java 8 before we can load PySpark functions. 
'''
/usr/libexec/java_home -V
export JAVA_HOME=/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home
export PATH=$JAVA_HOME/bin:$PATH
which java
'''

'\n/usr/libexec/java_home -V\nexport JAVA_HOME=/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home\nexport PATH=$JAVA_HOME/bin:$PATH\nwhich java\n'

In [10]:
# Load libraries
import os
import pyspark
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

In [11]:
# Set working directory
os.chdir('/Users/amawest/Desktop')

In [12]:
!java -version

openjdk version "1.8.0_292"
OpenJDK Runtime Environment (AdoptOpenJDK)(build 1.8.0_292-b10)
OpenJDK 64-Bit Server VM (AdoptOpenJDK)(build 25.292-b10, mixed mode)


In [13]:
# load in all the raw data from Virginia Open Court Data
data_2010 = pd.read_csv('circuit_data/circuit_criminal_2010_anon_00.csv')
data_2011 = pd.read_csv('circuit_data/circuit_criminal_2011_anon_00.csv')
data_2012 = pd.read_csv('circuit_data/circuit_criminal_2012_anon_00.csv')
data_2013 = pd.read_csv('circuit_data/circuit_criminal_2013_anon_00.csv')
data_2014 = pd.read_csv('circuit_data/circuit_criminal_2014_anon_00.csv')
data_2015 = pd.read_csv('circuit_data/circuit_criminal_2015_anon_00.csv')
data_2016 = pd.read_csv('circuit_data/circuit_criminal_2016_anon_00.csv')
data_2017 = pd.read_csv('circuit_data/circuit_criminal_2017_anon_00.csv')
data_2018 = pd.read_csv('circuit_data/circuit_criminal_2018_anon_00.csv')
data_2019 = pd.read_csv('circuit_data/circuit_criminal_2019_anon_00.csv')

In [27]:
# reorder so that all dataframes are organzed by "OffenseDate" (when the offense was supposedly committed)
data_2010['OffenseDate'] = pd.to_datetime(data_2010['OffenseDate'], format='%Y-%m-%d', errors='ignore')
data_2010 = data_2010.sort_values(by=['OffenseDate'])
data_2010.to_csv('circuit_data/circuit_criminal_2010_anon_00.csv')

data_2011['OffenseDate'] = pd.to_datetime(data_2011['OffenseDate'], format='%Y-%m-%d', errors='ignore')
data_2011 = data_2011.sort_values(by=['OffenseDate'])
data_2011.to_csv('circuit_data/circuit_criminal_2011_anon_00.csv')

data_2012['OffenseDate'] = pd.to_datetime(data_2012['OffenseDate'], format='%Y-%m-%d', errors='ignore')
data_2012 = data_2012.sort_values(by=['OffenseDate'])
data_2012.to_csv('circuit_data/circuit_criminal_2012_anon_00.csv')

data_2013['OffenseDate'] = pd.to_datetime(data_2013['OffenseDate'], format='%Y-%m-%d', errors='ignore')
data_2013 = data_2013.sort_values(by=['OffenseDate'])
data_2013.to_csv('circuit_data/circuit_criminal_2013_anon_00.csv')

data_2014['OffenseDate'] = pd.to_datetime(data_2014['OffenseDate'], format='%Y-%m-%d', errors='ignore')
data_2014 = data_2014.sort_values(by=['OffenseDate'])
data_2014.to_csv('circuit_data/circuit_criminal_2014_anon_00.csv')

data_2015['OffenseDate'] = pd.to_datetime(data_2015['OffenseDate'], format='%Y-%m-%d', errors='ignore')
data_2015 = data_2015.sort_values(by=['OffenseDate'])
data_2015.to_csv('circuit_data/circuit_criminal_2015_anon_00.csv')

data_2016['OffenseDate'] = pd.to_datetime(data_2016['OffenseDate'], format='%Y-%m-%d', errors='ignore')
data_2016 = data_2016.sort_values(by=['OffenseDate'])
data_2016.to_csv('circuit_data/circuit_criminal_2016_anon_00.csv')

data_2017['OffenseDate'] = pd.to_datetime(data_2017['OffenseDate'], format='%Y-%m-%d', errors='ignore')
data_2017 = data_2017.sort_values(by=['OffenseDate'])
data_2017.to_csv('circuit_data/circuit_criminal_2017_anon_00.csv')

data_2018['OffenseDate'] = pd.to_datetime(data_2018['OffenseDate'], format='%Y-%m-%d', errors='ignore')
data_2018 = data_2018.sort_values(by=['OffenseDate'])
data_2018.to_csv('circuit_data/circuit_criminal_2018_anon_00.csv')

data_2019['OffenseDate'] = pd.to_datetime(data_2019['OffenseDate'], format='%Y-%m-%d', errors='ignore')
data_2019 = data_2019.sort_values(by=['OffenseDate'])
data_2019.to_csv('circuit_data/circuit_criminal_2019_anon_00.csv')

## Part 2: Match Counties to Zipcodes

In [14]:
correct_county_zips = pd.read_csv('county_zip_edited.csv')
correct_county_zips

,Zip Code,County
0,20108,Manassas city
1,20110,Manassas city
2,20113,Manassas Park city
3,22030,Fairfax city
4,22038,Fairfax city
...,...,...
2094,26855,Grant County
2095,26865,Hampshire County
2096,26866,Pendleton County
2097,26884,Pendleton County


In [16]:
# split out zipcodes in the file and make sure every thing is correctly in the state of Virginia
for i in range(0,10):
    file = pd.read_csv(f'circuit_data/circuit_criminal_201{i}_anon_00.csv')
    file[['city_state', 'Zip Code']] = file.Address.str.rsplit(" ", n=1, expand=True,)
    file = file[file['city_state'].notnull()]
    file = file[file['city_state'].str.contains('VA')]
    #file_zips = file_zips[~file_zips['Zip Code'].str.contains('-')]
    #file_zips = file_zips[file_zips['Zip Code'].str.contains('(?<!\n)[\d]{5,6}[\-]?[\d]*')]
    #file['state']  = 'Virginia'
    correct_county_zips['Zip Code'] = correct_county_zips['Zip Code'].astype(str)
    file['Zip Code'] = file['Zip Code'].astype(str)
    file = pd.merge(file, correct_county_zips, how='left', on='Zip Code')
    file.to_csv(f'circuit_data/zips_circuit_criminal_201{i}_anon_00.csv', index=False)

/Users/amawest/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5,27,39) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/amawest/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (39,42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Part 3: Aggregate Data from Individuals --> % of Various Populations
- The FIPS covered less than counties, so I'd like to try counties and see how much coverage we see. 

In [47]:
x = pd.read_csv('circuit_data/zips_circuit_criminal_2010_anon_00.csv')

In [52]:
len(data_2010.fips.unique())

118

In [51]:
len(x.County.unique())

149

In [59]:
# Read in files
# note that we only keep DISTINCT (i.e. unique) person IDs from here on out
for i in range(0,10):
    infile = (f'circuit_data/zips_circuit_criminal_201{i}_anon_00.csv')
    spark = SparkSession \
        .builder \
        .appName("Racial Disparity") \
        .getOrCreate()
    circuit = spark.read.csv(infile, inferSchema = True, header = True)
    circuit.createOrReplaceTempView("data")
    sqlDF = spark.sql(
        
        # currently set to look at marijuana code sections only
        # but just take out code sections to see everything (or add more code sections)
        '''
        SELECT COUNT(DISTINCT(person_id)) as Count, Race, Sex, County FROM data
                   WHERE (CodeSection LIKE '18.2-248%' 
                   OR CodeSection LIKE '18.2-250%'
                   OR CodeSection LIKE '18.2-255%')
                   GROUP BY Race, County, Sex
                   ORDER BY Count DESC
                   '''
    )
    data = sqlDF.toPandas()
    data['Race and Sex'] = data['Race'] + ' - ' + data['Sex']
    data = data[['County', 'Race and Sex', 'Count']]
    df_p = data.pivot_table(index=['County'], columns=['Race and Sex'], values='Count', aggfunc=np.sum)
    data_1 = df_p.fillna(0)
    data_1.to_csv(f'circuit_data/201{i}_finished.csv')

# ================
# NOTE: Add/remove as desired to change the type of query
# Marijuana
# WHERE (CodeSection LIKE '18.2-248%' 
# OR CodeSection LIKE '18.2-250%'
# OR CodeSection LIKE '18.2-255%')

# Felonys
# AND (ChargeType == 'Felony')

In [70]:
# Some code to rename 2019 to match 2010-2018 column labels (currently not using)
x = pd.read_csv('circuit_data/2019_finished.csv')
new_cols = {'American Indian Or Alaskan Native - Female':         'American Indian - Female',
            'American Indian Or Alaskan Native - Male':           'American Indian - Male',
            'Asian Or Pacific Islander - Female':                 'Asian Or Pacific Islander - Female',
            'Asian Or Pacific Islander - Male':                   'Asian Or Pacific Islander - Male',
            'Black - Female':                                     'Black (Non-Hispanic) - Female',
            'Black - Male':                                       'Black (Non-Hispanic) - Male',
            'Hispanic - Female':                                  'Hispanic - Female',
            'Hispanic - Male':                                    'Hispanic - Male',
            'White - Female':                                     'White Caucasian (Non-Hispanic) - Female',
            'White - Male':                                       'White Caucasian (Non-Hispanic) - Male',
            'Other (Includes Not Applicable, Unknown) - Female':  'Other (Includes Not Applicable, Unknown) - Female',
            'Other (Includes Not Applicable, Unknown) - Male':    'Other (Includes Not Applicable, Unknown) - Male',
           }
x.rename(columns=new_cols,
          inplace=True)
x.to_csv('circuit_data/2019_finished.csv', index=None)

In [86]:
len(pd.read_csv(infile).CTYNAME.unique())

133

In [71]:
for i in range(0,10):
    # Note: make sure to reset the year each time
    infile = 'census_data_fips.csv'
    spark = SparkSession \
    .builder \
    .appName("Census Data") \
    .getOrCreate()
    circuit = spark.read.csv(infile, inferSchema=True, header = True)
    circuit.createOrReplaceTempView("census_data")
    sqlDF = spark.sql(f'SELECT * \
                   FROM census_data \
                   WHERE Year = 201{i}')
    census = sqlDF.toPandas()
    
    #--------------------#
    data = pd.read_csv(f'circuit_data/201{i}_finished.csv')
    #--------------------#
    data['Total_Crimes'] = 0
    data['Total_Crimes'] = data.sum(axis=1) 
    data.head(2)
    #del census['Year']
    census.head(2)
    result = pd.merge(census,
                  data,
                  left_on = 'CTYNAME',
                  right_on= 'County', 
                  how='left')
    result.head(2)
    data.columns
    result['Percent_Overall']          = (result['Total_Crimes']   / result['Total'])*100
    result['Percent_White_Male']       = (result['White Caucasian (Non-Hispanic) - Male']   / result['White_Male'])*100
    result['Percent_White_Female']     = (result['White Caucasian (Non-Hispanic) - Female'] / result['White_Female'])*100
    result['Percent_Black_Male']       = (result['Black (Non-Hispanic) - Male']             / result['Black_Male'])*100
    result['Percent_Black_Female']     = (result['Black (Non-Hispanic) - Female']           / result['Black_Female'])*100
    #result['Percent_Nat_Amer_Male']    = (result['American Indian - Male']                  / result['Native_Amer_Male'])*100
    #result['Percent_Nat_Amer_Female']  = (result['American Indian - Female']                / result['Native_Amer_Female'])*100
    result['Percent_Hispanic_Male']    = (result['Hispanic - Male']                         / result['Hispanic_Male'])*100
    result['Percent_Hispanic_Female']  = (result['Hispanic - Female']                       / result['Hispanic_Female'])*100
    result['Percent_Asian_Pac_Male']   = (result['Asian Or Pacific Islander - Male']        / result['Asian_Male'])*100
    result['Percent_Asian_Pac_Female'] = (result['Asian Or Pacific Islander - Female']      / result['Asian_Female'])*100
    
    result['Percent_Overall']            = round(result['Percent_Overall'],3)
    result['Disparity_White_Male']       = round(result['Percent_White_Male'] - result['Percent_Overall'],3)
    result['Disparity_White_Female']     = round(result['Percent_White_Female'] - result['Percent_Overall'],3)
    result['Disparity_Black_Male']       = round(result['Percent_Black_Male'] - result['Percent_Overall'],3)
    result['Disparity_Black_Female']     = round(result['Percent_Black_Female'] - result['Percent_Overall'],3)
    #result['Disparity_Nat_Amer_Male']    = round(result['Percent_Nat_Amer_Male'] - result['Percent_Overall'],3)
    #result['Disparity_Nat_Amer_Female']  = round(result['Percent_Nat_Amer_Female'] - result['Percent_Overall'],3)
    result['Disparity_Hispanic_Male']    = round(result['Percent_Hispanic_Male'] - result['Percent_Overall'],3)
    result['Disparity_Hispanic_Female']  = round(result['Percent_Hispanic_Female'] - result['Percent_Overall'],3)
    result['Disparity_Asian_Pac_Male']   = round(result['Percent_Asian_Pac_Male'] - result['Percent_Overall'],3)
    result['Disparity_Asian_Pac_Female'] = round(result['Percent_Asian_Pac_Female'] - result['Percent_Overall'],3)
    
    result = result[['fips', 
                     'CTYNAME',
                     'Percent_Overall',
                     'Disparity_White_Male',
                     'Disparity_White_Female',
                     'Disparity_Black_Male',
                     'Disparity_Black_Female', 
                     #'Disparity_Nat_Amer_Male',
                     #'Disparity_Nat_Amer_Female', 
                     'Disparity_Hispanic_Male',
                     'Disparity_Hispanic_Female',
                     'Disparity_Asian_Pac_Male',
                     'Disparity_Asian_Pac_Female'
        ]]
    result
    #--------------------#
    result.to_csv(f'circuit_data/Final/201{i}_disparities.csv',index=None)
    #--------------------#

## Part 4: Put It All Together & Finalize

In [72]:
x_2010 = pd.read_csv('circuit_data/Final/2010_disparities.csv')
x_2011 = pd.read_csv('circuit_data/Final/2011_disparities.csv')
x_2012 = pd.read_csv('circuit_data/Final/2012_disparities.csv')
x_2013 = pd.read_csv('circuit_data/Final/2013_disparities.csv')
x_2014 = pd.read_csv('circuit_data/Final/2014_disparities.csv')
x_2015 = pd.read_csv('circuit_data/Final/2015_disparities.csv')
x_2016 = pd.read_csv('circuit_data/Final/2016_disparities.csv')
x_2017 = pd.read_csv('circuit_data/Final/2017_disparities.csv')
x_2018 = pd.read_csv('circuit_data/Final/2018_disparities.csv')
x_2019 = pd.read_csv('circuit_data/Final/2019_disparities.csv')

In [73]:
x_2010['year'] = 2010
x_2011['year'] = 2011
x_2012['year'] = 2012
x_2013['year'] = 2013
x_2014['year'] = 2014
x_2015['year'] = 2015
x_2016['year'] = 2016
x_2017['year'] = 2017
x_2018['year'] = 2018
x_2019['year'] = 2019

In [74]:
x_2010.to_csv('circuit_data/Final/2010_disparities.csv')
x_2011.to_csv('circuit_data/Final/2011_disparities.csv')
x_2012.to_csv('circuit_data/Final/2012_disparities.csv')
x_2013.to_csv('circuit_data/Final/2013_disparities.csv')
x_2014.to_csv('circuit_data/Final/2014_disparities.csv')
x_2015.to_csv('circuit_data/Final/2015_disparities.csv')
x_2016.to_csv('circuit_data/Final/2016_disparities.csv')
x_2017.to_csv('circuit_data/Final/2017_disparities.csv')
x_2018.to_csv('circuit_data/Final/2018_disparities.csv')
x_2019.to_csv('circuit_data/Final/2019_disparities.csv')

In [75]:
files = ['circuit_data/Final/2010_disparities.csv',
         'circuit_data/Final/2011_disparities.csv',
         'circuit_data/Final/2012_disparities.csv',
         'circuit_data/Final/2013_disparities.csv',
         'circuit_data/Final/2014_disparities.csv',
         'circuit_data/Final/2015_disparities.csv',
         'circuit_data/Final/2016_disparities.csv',
         'circuit_data/Final/2017_disparities.csv',
         'circuit_data/Final/2018_disparities.csv',
         'circuit_data/Final/2019_disparities.csv'
         ]

In [76]:
dfs = [pd.read_csv(f, sep=",") for f in files]
all_files = pd.concat(dfs,ignore_index=True)

In [77]:
del all_files['Unnamed: 0']

In [78]:
all_files

,fips,CTYNAME,Percent_Overall,Disparity_White_Male,Disparity_White_Female,Disparity_Black_Male,Disparity_Black_Female,Disparity_Nat_Amer_Male,Disparity_Hispanic_Male,Disparity_Hispanic_Female,Disparity_Asian_Pac_Male,Disparity_Asian_Pac_Female,year
0,1,Accomack County,0.139,-0.042,-0.130,0.483,-0.000,-0.139,-0.139,-0.139,-0.139,-0.139,2010
1,3,Albemarle County,0.057,0.004,-0.048,0.390,0.021,-0.057,0.080,-0.057,-0.057,-0.057,2010
2,5,Alleghany County,0.191,0.052,-0.139,1.637,0.347,-0.191,-0.191,-0.191,-0.191,-0.191,2010
3,7,Amelia County,0.212,0.019,-0.128,0.671,-0.212,-0.212,-0.212,-0.212,-0.212,-0.212,2010
4,9,Amherst County,0.173,0.028,-0.112,0.588,-0.141,-0.173,-0.173,-0.173,-0.173,-0.173,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1325,800,Suffolk city,0.150,-0.030,-0.100,0.315,-0.102,-0.150,-0.150,-0.150,-0.024,-0.150,2019
1326,810,Virginia Beach city,0.144,0.037,-0.065,0.343,-0.070,-0.144,-0.144,-0.144,-0.144,-0.133,2019
1327,820,Waynesboro city,0.751,0.001,-0.150,2.033,-0.377,-0.751,-0.649,-0.751,-0.751,-0.751,2019
1328,830,Williamsburg city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019


In [79]:
all_files.to_csv('marijuana_disparities_final.csv')